## 深入理解训练流程

## 1.1 数据加载机制（优先）
**重点文件：**
- flmm/datasets/png.py - PNG 数据集
- flmm/datasets/transforms.py - 数据转换
- configs/deepseek_vl/frozen_deepseek_vl_1_3b_chat_unet_sam_l_refcoco_png.py (数据集配置部分)
学习要点：
- 如何加载图像和标注
- prompt 模板如何构建
- 数据增强和预处理流程
- batch 组织方式

In [1]:

%load_ext autoreload
%autoreload 2

In [2]:

# 设置使用 3 号 GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ[ 'HF_ENDPOINT'] ="https://hf-mirror.com"
# # 你的代理地址
# proxy_url = "http://10.156.216.30:16371" 

# # 设置 HTTP 和 HTTPS 代理
# os.environ['http_proxy'] = proxy_url
# os.environ['https_proxy'] = proxy_url
# os.environ['HTTP_PROXY'] = proxy_url # 建议同时设置大小写
# os.environ['HTTPS_PROXY'] = proxy_url
# 检查环境
import sys
print(f"Python: {sys.version}")

try:
    import torch
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU 数量: {torch.cuda.device_count()}")
        print(f"当前 GPU: {torch.cuda.current_device()}")
        print(f"GPU 名称: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("PyTorch not installed!")

try:
    import transformers
    print(f"Transformers: {transformers.__version__}")
except ImportError:
    print("Transformers not installed!")


Python: 3.10.19 (main, Oct 21 2025, 16:43:05) [GCC 11.2.0]
PyTorch: 2.2.2+cu118
CUDA available: True
GPU 数量: 1
当前 GPU: 0
GPU 名称: NVIDIA A800 80GB PCIe
Transformers: 4.51.3


In [3]:
import torch
from flmm.datasets.png import PNGDataset
from mmengine.config import Config
from xtuner.registry import BUILDER

# 加载配置
cfg = Config.fromfile('configs/qwen/frozen_qwen2_5_vl_7b_instruct_unet_sam_l_refcoco_png.py')
# cfg = Config.fromfile('configs/deepseek_vl/frozen_deepseek_vl_1_3b_chat_unet_sam_l_refcoco_png.py')
# 从配置构建数据集对象（不是配置字典）
dataset = BUILDER.build(cfg.train_dataloader.dataset)

/home/cvprtemp/miniconda3/envs/flmm-qwen-py310/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/cvprtemp/miniconda3/envs/flmm-qwen-py310/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/home/cvprtemp/miniconda3/envs/flmm-qwen-py310/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/home/cvprtemp/miniconda3/envs/flmm-qwen-py310/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/home/cvprtemp/miniconda3/envs/flmm-qwen-py310/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/home/cvprtemp/miniconda3/envs/flmm-qwen-py310/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


loading annotations into memory...
Done (t=3.97s)
creating index...
index created!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Wrapping Qwen processor with QwenImageProcessorWrapper
Image token ids: [151655], decoded: <|image_pad|>
Wrapping Qwen processor with QwenImageProcessorWrapper
Image token ids: [151655], decoded: <|image_pad|>
Wrapping Qwen processor with QwenImageProcessorWrapper
Image token ids: [151655], decoded: <|image_pad|>
Wrapping Qwen processor with QwenImageProcessorWrapper
Image token ids: [151655], decoded: <|image_pad|>


In [12]:
# 检查数据集
print(f"数据集大小: {len(dataset)}")

# 获取第一个样本
if len(dataset) > 0:
    data_sample = dataset[0]
    print(data_sample.keys())
    print("\n第一个样本的键:")
    for key, value in data_sample.items():
        if isinstance(value, torch.Tensor):
            print(f"  {key}: Tensor shape {value.shape}, dtype {value.dtype}")
        elif isinstance(value, (list, tuple)):
            print(f"  {key}: {type(value).__name__} of length {len(value)}")
        else:
            print(f"  {key}: {type(value).__name__} = {value}")

数据集大小: 190157
In QwenImageProcessorWrapper, result["image_grid_thw"].shape is 
 torch.Size([1, 3])
In PNGDataset, image_grid_thw is 
 [[1, 30, 46]]
In PNGDataset, result_dict["image_grid_thw"].shape is 
 torch.Size([1, 3])
dict_keys(['input_ids', 'mask_ids', 'pixel_values', 'padded_masks', 'masks', 'gt_masks', 'image_sizes', 'mask_infos', 'image', 'file_name', 'meta_data', 'labels', 'image_grid_thw'])

第一个样本的键:
  input_ids: Tensor shape torch.Size([390]), dtype torch.int64
  mask_ids: Tensor shape torch.Size([390]), dtype torch.int64
  pixel_values: Tensor shape torch.Size([1380, 1176]), dtype torch.float32
  padded_masks: Tensor shape torch.Size([2, 420, 644]), dtype torch.uint8
  masks: Tensor shape torch.Size([2, 420, 644]), dtype torch.uint8
  gt_masks: Tensor shape torch.Size([2, 426, 640]), dtype torch.uint8
  image_sizes: Tensor shape torch.Size([2]), dtype torch.int64
  mask_infos: list of length 2
  image: JpegImageFile = <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=

## 数据流图
0. 原始 PNG 数据<br>
  ↓
1. 加载图像和分割图<br>
  ↓
2. Tokenize prompt + captions → input_ids<br>
  ↓
3. 构建 mask_ids（标记每个 token 属于哪个 mask）<br>
  ↓
4. 图像预处理 → pixel_values (384×384)<br>
  ↓
5. 提取和调整 mask → masks (255×384)<br>
  ↓
6. Padding mask → padded_masks (384×384)<br>
  ↓
7. 保存原始尺寸 → gt_masks (426×640)<br>
  ↓
8. 构建 labels（用于语言建模）<br>
  ↓<br>
输出 data_sample 字典<br>

## 2.2 模型结构理解

**重点文件：**
- `flmm/models/frozen_deepseek_vl.py` - DeepSeekVL 冻结模型
- `flmm/models/mask_head/mask_decoder.py` - U-Net 分割头
- `flmm/models/mask_head/mask_refiner.py` - SAM 包装器

**学习要点：**
- 如何冻结基础 LMM 参数
- 如何提取 hidden states 和 attentions
- 注意力如何映射到图像空间
- U-Net 和 SAM 如何集成
- 损失计算方式（Dice Loss + CrossEntropy Loss）

**对比学习：**
- 查看 `frozen_llava.py` 和 `frozen_mgm.py`，了解不同模型的适配方式

**详细文档：** 参见 `doc/MODEL_STRUCTURE.md`

In [ ]:
# 2.2.2 理解注意力提取过程（不加载模型版本）
import torch

# 获取一个数据样本
data_sample = dataset[0]

# 从 tokenizer 获取图像 token 索引
tokenizer = BUILDER.build(cfg.tokenizer)
image_token_idx = tokenizer.encode('<image_placeholder>', add_special_tokens=False)[-1]

# 从配置获取 clip_shape（DeepSeekVL 固定为 24）
clip_shape = 24  # DeepSeekVL 使用 24×24 的特征图

print("=" * 60)
print("注意力提取流程")
print("=" * 60)

print("\n1. 输入准备:")
print(f"   input_ids shape: {data_sample['input_ids'].shape}")
print(f"   pixel_values shape: {data_sample['pixel_values'].shape}")
print(f"   mask_ids shape: {data_sample['mask_ids'].shape}")

print("\n2. 图像 token 位置:")
images_seq_mask = data_sample['input_ids'] == image_token_idx
num_image_tokens = images_seq_mask.sum().item()
image_token_positions = torch.where(images_seq_mask)[0].tolist()
print(f"   图像 token 索引: {image_token_idx}")
print(f"   图像 token 数量: {num_image_tokens}")
print(f"   图像 token 位置: {image_token_positions[:10]}... (共 {len(image_token_positions)} 个)")

print("\n3. 注意力重塑:")
print(f"   clip_shape: {clip_shape}×{clip_shape} = {clip_shape*clip_shape} 个图像 patch")
print(f"   注意力权重将重塑为: [num_heads, seq_len, {clip_shape}, {clip_shape}]")
print(f"   每个图像 token 对应一个 {clip_shape}×{clip_shape} 的特征图位置")

print("\n4. Mask 分组:")
masks = data_sample['masks']
print(f"   mask 数量: {len(masks)}")
for mask_id in range(len(masks)):
    matched = data_sample['mask_ids'] == mask_id
    matched_count = matched.sum().item()
    matched_positions = torch.where(matched)[0].tolist()
    print(f"   Mask {mask_id}: {matched_count} 个 token")
    print(f"      Token 位置: {matched_positions[:5]}..." if matched_count > 5 else f"      Token 位置: {matched_positions}")

print("\n5. 注意力提取过程（概念性）:")
print("   a) 前向传播 LMM，获取所有层的 attention weights")
print("   b) 提取 attention[:, :, image_token_positions] → [num_heads, seq_len, num_image_tokens]")
print("   c) Reshape: [num_heads, seq_len, num_image_tokens] → [num_heads, seq_len, 24, 24]")
print("   d) 对于每个 mask，提取对应 token 的注意力")
print("   e) Merge heads: [num_heads, H, W] → [H, W] (mean/max)")
print("   f) Concat layers: [num_layers, H, W] → [num_layers*num_heads, H, W]")

In [ ]:
print(masks.shape)
print(masks)
print(mask_id)
print(data_sample['mask_ids'].shape)
print(data_sample['mask_ids'])